In [1]:
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
class Model:
    def __init__(self, k):
        self.k = k
        self.X = None
        self.y = None
        self.n_classes = None

    def fit(self, X, y):
        self.X = X
        self.y = y
        self.n_classes = np.unique(y).size

    def kernel(self, r):
        return np.exp(-0.5 * (r ** 2)) / np.sqrt(2 * np.pi)

    def predict_proba(self, X):
        results = []
        distances = cdist(X, self.X)
        radiuses = np.sort(distances, axis=1)[:, self.k + 1]
        for i in range(distances.shape[0]):
            dist_row = distances[i]
            radius = radiuses[i]
            indexes = np.where(dist_row < radius)
            distances_lower = dist_row[indexes]
            k = self.kernel(distances_lower / radius)
            labels = self.y[indexes]
            probas = np.zeros(self.n_classes)
            for c in range(self.n_classes):
                if (labels == c).sum() > 0:
                    probas[c] = k[labels == c].sum()
            probas /= probas.sum()
            results.append(probas)
        return results

    def predict(self, X):
        probas = self.predict_proba(X)
        return np.argmax(probas, axis=1)

In [3]:
data = load_iris()

X = data.data
y = data.target

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [4]:
model = Model(3)
model.fit(train_X, train_y)

In [5]:
accuracy_score(test_y, model.predict(test_X))

1.0